In [1]:
# check nvcc version
!nvcc -V
# check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
gcc (GCC) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
!nvidia-smi

Tue Nov 10 14:18:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:3B:00.0 Off |                    0 |
| 33%   55C    P2   242W / 260W |  13905MiB / 45553MiB |     73%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:5E:00.0 Off |                    0 |
| 33%   20C    P8    10W / 260W |      0MiB / 45553MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

In [3]:
! export CUDA_VISIBLE_DEVICES=1,2

In [4]:
!pwd

/home/lbz/ML


In [5]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [6]:
train_features = pd.read_csv('lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('lish-moa/test_features.csv')
sample_submission = pd.read_csv('lish-moa/sample_submission.csv')

In [7]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [8]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [9]:
train_targets_scored.sum()[1:].sort_values()

atp-sensitive_potassium_channel_antagonist      1
erbb2_inhibitor                                 1
diuretic                                        6
autotaxin_inhibitor                             6
protein_phosphatase_inhibitor                   6
                                             ... 
serotonin_receptor_antagonist                 404
dopamine_receptor_antagonist                  424
cyclooxygenase_inhibitor                      435
proteasome_inhibitor                          726
nfkb_inhibitor                                832
Length: 206, dtype: object

In [10]:
train_features['cp_type'].unique()

array(['trt_cp', 'ctl_vehicle'], dtype=object)

## PCA features + Existing features¶

In [11]:
# GENES
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [12]:
#CELLS
n_comp = 15

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

## feature Selection using Variance Encoding¶

In [13]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(threshold=0.5)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features

,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,...,917,918,919,920,921,922,923,924,925,926
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,-0.450285,-0.176778,-1.262943,0.219107,-0.890670,0.393604,-0.703376,-0.615139,0.174407,0.082941
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0.063234,0.658824,0.429385,-0.226422,0.271831,0.863835,0.003597,0.669397,0.447651,1.207365
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.115802,0.726273,-0.212644,-0.902482,-0.118799,-0.336548,0.015536,0.572233,-0.261651,-0.638141
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,0.590366,0.698760,0.050321,-0.793301,0.295411,0.147857,0.056161,0.689218,-1.433683,1.323147
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,-0.000223,-0.287454,-0.110246,-0.105291,-0.396913,0.090983,-0.211590,0.350304,-0.326626,-0.344389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,-0.492270,0.802396,0.332499,-0.204876,0.238577,-0.483204,0.585078,0.173586,-0.611718,1.607084
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,-1.364079,-0.375444,-1.433534,-0.858483,1.072457,0.101450,0.435098,-0.219500,0.377156,0.555680
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,-0.511130,-0.035609,-0.310135,-0.166686,-0.458886,-0.003948,0.292592,0.331622,-0.006669,0.081750
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-1.129357,0.020524,-0.043233,-0.440007,0.302835,0.776086,-1.737516,-0.531532,-0.351892,0.542268


In [14]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [15]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [16]:
train

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,-0.1166,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0,0,0,0,0,0,0,0,0,0


## Other 

In [17]:
#Binning
# for col in GENES:
#     train.loc[:, f'{col}_bin'] = pd.cut(train[col], bins=3, labels=False)
#     test.loc[:, f'{col}_bin'] = pd.cut(test[col], bins=3, labels=False)

In [18]:
#Distribution plots
# plt.figure(figsize=(16,16))
# sns.set_style("whitegrid")

# gene_choice = np.random.choice(len(GENES), 16)
# for i, col in enumerate(gene_choice):
#     plt.subplot(4, 4, i+1)
#     plt.hist(train_features.loc[:, GENES[col]],bins=100, color='orange')
#     plt.title(GENES[col])

In [19]:
#[Naive] Outlier Removal¶
# train_ = train.copy() [Didn't wanted to actually normalize, so created a copy and normalized that for further calculation]
# for col in GENES:
    
# #     train_[col] = (train[col]-np.mean(train[col])) / (np.std(train[col]))
    
#     mean = train_[col].mean()
#     std = train_[col].std()

#     std_r = mean + 4*std
#     std_l = mean - 4*std

#     drop = train_[col][(train_[col]>std_r) | (train_[col]<std_l)].index.values

# train = train.drop(drop).reset_index(drop=True)
# # folds = folds.drop(drop).reset_index(drop=True)
# target = target.drop(drop).reset_index(drop=True)

## PCA

In [20]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

## CV folds

In [21]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=5)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,0,0,0,0,0,0,0,0,0,1
3,id_0015fd391,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,0,0,0,0,0,0,0,0,0,2
4,id_001626bd3,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,-0.1166,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0,0,0,0,0,0,0,0,0,4
21945,id_fffb70c0c,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0,0,0,0,0,0,0,0,0,1


In [22]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1136)
(21948, 1137)
(3624, 930)
(21948, 207)
(3982, 207)


## Dataset Classes

In [23]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [24]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   

## Model

In [25]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

## Preprocessing steps

In [26]:
def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
#     data.loc[:, 'cp_time'] = data.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
#     data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})

# --------------------- Normalize ---------------------
#     for col in GENES:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#     for col in CELLS:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#--------------------- Removing Skewness ---------------------
#     for col in GENES + CELLS:
#         if(abs(data[col].skew()) > 0.75):
            
#             if(data[col].skew() < 0): # neg-skewness
#                 data[col] = data[col].max() - data[col] + 1
#                 data[col] = np.sqrt(data[col])
            
#             else:
#                 data[col] = np.sqrt(data[col])
    
    return data

In [27]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)


932

In [34]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

## Single fold training

In [29]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [30]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [35]:
# Averaging on multiple SEEDS
import torch
import torch.utils.data
import torch.optim.lr_scheduler #import OneCycleLR
SEED = [0, 1, 2, 3 ,4, 5]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

FOLD: 0, EPOCH: 0, train_loss: 0.5443700503126003
FOLD: 0, EPOCH: 0, valid_loss: 0.03568510147077697
FOLD: 0, EPOCH: 1, train_loss: 0.023244795254499153
FOLD: 0, EPOCH: 1, valid_loss: 0.02035433878856046
FOLD: 0, EPOCH: 2, train_loss: 0.019910719556113083
FOLD: 0, EPOCH: 2, valid_loss: 0.01823332668947322
FOLD: 0, EPOCH: 3, train_loss: 0.01868445028051518
FOLD: 0, EPOCH: 3, valid_loss: 0.017551363365990776
FOLD: 0, EPOCH: 4, train_loss: 0.01799021893437358
FOLD: 0, EPOCH: 4, valid_loss: 0.017370463082832948
FOLD: 0, EPOCH: 5, train_loss: 0.01758874756405535
FOLD: 0, EPOCH: 5, valid_loss: 0.017232250182756356
FOLD: 0, EPOCH: 6, train_loss: 0.01751608162632455
FOLD: 0, EPOCH: 6, valid_loss: 0.017093399566199097
FOLD: 0, EPOCH: 7, train_loss: 0.017356589726725782
FOLD: 0, EPOCH: 7, valid_loss: 0.017017428390681745
FOLD: 0, EPOCH: 8, train_loss: 0.017436791218115366
FOLD: 0, EPOCH: 8, valid_loss: 0.017046274989843367
FOLD: 0, EPOCH: 9, train_loss: 0.017337836095712322
FOLD: 0, EPOCH: 9, va

FOLD: 3, EPOCH: 4, train_loss: 0.017925005460130997
FOLD: 3, EPOCH: 4, valid_loss: 0.01764894774449723
FOLD: 3, EPOCH: 5, train_loss: 0.01759953796863556
FOLD: 3, EPOCH: 5, valid_loss: 0.017380821172680173
FOLD: 3, EPOCH: 6, train_loss: 0.017458208384451227
FOLD: 3, EPOCH: 6, valid_loss: 0.017402460346264498
FOLD: 3, EPOCH: 7, train_loss: 0.017422814239356398
FOLD: 3, EPOCH: 7, valid_loss: 0.017321541027298996
FOLD: 3, EPOCH: 8, train_loss: 0.017293883096156776
FOLD: 3, EPOCH: 8, valid_loss: 0.017337085918656417
FOLD: 3, EPOCH: 9, train_loss: 0.017392934158282435
FOLD: 3, EPOCH: 9, valid_loss: 0.017390202145491328
FOLD: 3, EPOCH: 10, train_loss: 0.017363358673680086
FOLD: 3, EPOCH: 10, valid_loss: 0.01727876527501004
FOLD: 3, EPOCH: 11, train_loss: 0.017347375457377537
FOLD: 3, EPOCH: 11, valid_loss: 0.017128254739301546
FOLD: 3, EPOCH: 12, train_loss: 0.01727951333547632
FOLD: 3, EPOCH: 12, valid_loss: 0.017154736018606596
FOLD: 3, EPOCH: 13, train_loss: 0.017142583764549614
FOLD: 3, 

FOLD: 1, EPOCH: 8, train_loss: 0.017343123733619417
FOLD: 1, EPOCH: 8, valid_loss: 0.017261562496423723
FOLD: 1, EPOCH: 9, train_loss: 0.017470213922037594
FOLD: 1, EPOCH: 9, valid_loss: 0.017129539618534702
FOLD: 1, EPOCH: 10, train_loss: 0.01739181028139116
FOLD: 1, EPOCH: 10, valid_loss: 0.01719156511660133
FOLD: 1, EPOCH: 11, train_loss: 0.01730385766891034
FOLD: 1, EPOCH: 11, valid_loss: 0.01707800121179649
FOLD: 1, EPOCH: 12, train_loss: 0.017271595683110798
FOLD: 1, EPOCH: 12, valid_loss: 0.016939546327505793
FOLD: 1, EPOCH: 13, train_loss: 0.017244966477965532
FOLD: 1, EPOCH: 13, valid_loss: 0.017027182132005693
FOLD: 1, EPOCH: 14, train_loss: 0.01715591472938009
FOLD: 1, EPOCH: 14, valid_loss: 0.01685487138373511
FOLD: 1, EPOCH: 15, train_loss: 0.016955674349235884
FOLD: 1, EPOCH: 15, valid_loss: 0.01680022225316082
FOLD: 1, EPOCH: 16, train_loss: 0.01686697063859606
FOLD: 1, EPOCH: 16, valid_loss: 0.01665883588471583
FOLD: 1, EPOCH: 17, train_loss: 0.016762978154356064
FOLD: 

FOLD: 4, EPOCH: 12, train_loss: 0.017327120709840372
FOLD: 4, EPOCH: 12, valid_loss: 0.017042480488973003
FOLD: 4, EPOCH: 13, train_loss: 0.017330871915201777
FOLD: 4, EPOCH: 13, valid_loss: 0.016985270673675196
FOLD: 4, EPOCH: 14, train_loss: 0.017136776603866314
FOLD: 4, EPOCH: 14, valid_loss: 0.01680439311478819
FOLD: 4, EPOCH: 15, train_loss: 0.01701876907136993
FOLD: 4, EPOCH: 15, valid_loss: 0.016832211879747254
FOLD: 4, EPOCH: 16, train_loss: 0.016972970015004925
FOLD: 4, EPOCH: 16, valid_loss: 0.01655214476798262
FOLD: 4, EPOCH: 17, train_loss: 0.016805244812174984
FOLD: 4, EPOCH: 17, valid_loss: 0.016512043428208147
FOLD: 4, EPOCH: 18, train_loss: 0.01655671655780811
FOLD: 4, EPOCH: 18, valid_loss: 0.01640384008309671
FOLD: 4, EPOCH: 19, train_loss: 0.016357804263901453
FOLD: 4, EPOCH: 19, valid_loss: 0.016225773548441274
FOLD: 4, EPOCH: 20, train_loss: 0.016059895626444748
FOLD: 4, EPOCH: 20, valid_loss: 0.016123654320836067
FOLD: 4, EPOCH: 21, train_loss: 0.01580359923747786

FOLD: 2, EPOCH: 16, train_loss: 0.01689943252806214
FOLD: 2, EPOCH: 16, valid_loss: 0.01659290875707354
FOLD: 2, EPOCH: 17, train_loss: 0.016716132973037336
FOLD: 2, EPOCH: 17, valid_loss: 0.016600101893501622
FOLD: 2, EPOCH: 18, train_loss: 0.016563245999640312
FOLD: 2, EPOCH: 18, valid_loss: 0.01636561727417367
FOLD: 2, EPOCH: 19, train_loss: 0.01626570352042715
FOLD: 2, EPOCH: 19, valid_loss: 0.016266629498984133
FOLD: 2, EPOCH: 20, train_loss: 0.016070854634154533
FOLD: 2, EPOCH: 20, valid_loss: 0.016160460880824498
FOLD: 2, EPOCH: 21, train_loss: 0.015766880454738504
FOLD: 2, EPOCH: 21, valid_loss: 0.016037146959986006
FOLD: 2, EPOCH: 22, train_loss: 0.015552471567323242
FOLD: 2, EPOCH: 22, valid_loss: 0.016020450847489494
FOLD: 2, EPOCH: 23, train_loss: 0.015412844683759022
FOLD: 2, EPOCH: 23, valid_loss: 0.015998252507831368
FOLD: 2, EPOCH: 24, train_loss: 0.015267864409564198
FOLD: 2, EPOCH: 24, valid_loss: 0.015945755770163876
FOLD: 3, EPOCH: 0, train_loss: 0.5439582876861095


FOLD: 0, EPOCH: 20, train_loss: 0.015982115222816017
FOLD: 0, EPOCH: 20, valid_loss: 0.01614499618964536
FOLD: 0, EPOCH: 21, train_loss: 0.01570065414694988
FOLD: 0, EPOCH: 21, valid_loss: 0.01604251497026001
FOLD: 0, EPOCH: 22, train_loss: 0.01539474224968665
FOLD: 0, EPOCH: 22, valid_loss: 0.016033572676990713
FOLD: 0, EPOCH: 23, train_loss: 0.01528168271255234
FOLD: 0, EPOCH: 23, valid_loss: 0.015975603567702428
FOLD: 0, EPOCH: 24, train_loss: 0.015232374500213326
FOLD: 0, EPOCH: 24, valid_loss: 0.015954404936305115
FOLD: 1, EPOCH: 0, train_loss: 0.5431977039122063
FOLD: 1, EPOCH: 0, valid_loss: 0.03615496318255152
FOLD: 1, EPOCH: 1, train_loss: 0.023613340788237427
FOLD: 1, EPOCH: 1, valid_loss: 0.020053368274654662
FOLD: 1, EPOCH: 2, train_loss: 0.020040924917312637
FOLD: 1, EPOCH: 2, valid_loss: 0.0184962909668684
FOLD: 1, EPOCH: 3, train_loss: 0.01869696517056529
FOLD: 1, EPOCH: 3, valid_loss: 0.017737968399056368
FOLD: 1, EPOCH: 4, train_loss: 0.01799754021635306
FOLD: 1, EPOCH

FOLD: 3, EPOCH: 24, train_loss: 0.015142982994354721
FOLD: 3, EPOCH: 24, valid_loss: 0.016215516626834868
FOLD: 4, EPOCH: 0, train_loss: 0.5450632183178179
FOLD: 4, EPOCH: 0, valid_loss: 0.033255747705698016
FOLD: 4, EPOCH: 1, train_loss: 0.0235478019044883
FOLD: 4, EPOCH: 1, valid_loss: 0.020098195225000382
FOLD: 4, EPOCH: 2, train_loss: 0.01999609199339065
FOLD: 4, EPOCH: 2, valid_loss: 0.01835161048386778
FOLD: 4, EPOCH: 3, train_loss: 0.01868024141109292
FOLD: 4, EPOCH: 3, valid_loss: 0.017774244397878648
FOLD: 4, EPOCH: 4, train_loss: 0.017977584470603346
FOLD: 4, EPOCH: 4, valid_loss: 0.01746804945703064
FOLD: 4, EPOCH: 5, train_loss: 0.01765168292204971
FOLD: 4, EPOCH: 5, valid_loss: 0.01722448045121772
FOLD: 4, EPOCH: 6, train_loss: 0.017456843821412844
FOLD: 4, EPOCH: 6, valid_loss: 0.01712947924222265
FOLD: 4, EPOCH: 7, train_loss: 0.017478533541324778
FOLD: 4, EPOCH: 7, valid_loss: 0.017085024608033045
FOLD: 4, EPOCH: 8, train_loss: 0.017373373509263216
FOLD: 4, EPOCH: 8, va

FOLD: 2, EPOCH: 3, train_loss: 0.0187435038185314
FOLD: 2, EPOCH: 3, valid_loss: 0.01771478575787374
FOLD: 2, EPOCH: 4, train_loss: 0.018032736003236925
FOLD: 2, EPOCH: 4, valid_loss: 0.017356336223227638
FOLD: 2, EPOCH: 5, train_loss: 0.01770238182844891
FOLD: 2, EPOCH: 5, valid_loss: 0.017265079436557633
FOLD: 2, EPOCH: 6, train_loss: 0.017511540963112013
FOLD: 2, EPOCH: 6, valid_loss: 0.017198951755251202
FOLD: 2, EPOCH: 7, train_loss: 0.017419401461771435
FOLD: 2, EPOCH: 7, valid_loss: 0.017113622490848815
FOLD: 2, EPOCH: 8, train_loss: 0.017379848657688803
FOLD: 2, EPOCH: 8, valid_loss: 0.01709895051483597
FOLD: 2, EPOCH: 9, train_loss: 0.01744803008151011
FOLD: 2, EPOCH: 9, valid_loss: 0.017165878335280077
FOLD: 2, EPOCH: 10, train_loss: 0.017418364150638597
FOLD: 2, EPOCH: 10, valid_loss: 0.017148912884294987
FOLD: 2, EPOCH: 11, train_loss: 0.017413903830869906
FOLD: 2, EPOCH: 11, valid_loss: 0.016967016804431165
FOLD: 2, EPOCH: 12, train_loss: 0.01730260394675576
FOLD: 2, EPOCH

FOLD: 0, EPOCH: 7, train_loss: 0.01745099469722397
FOLD: 0, EPOCH: 7, valid_loss: 0.01718003446502345
FOLD: 0, EPOCH: 8, train_loss: 0.017384537050257557
FOLD: 0, EPOCH: 8, valid_loss: 0.017146074399352073
FOLD: 0, EPOCH: 9, train_loss: 0.017378464030722778
FOLD: 0, EPOCH: 9, valid_loss: 0.017005011386104992
FOLD: 0, EPOCH: 10, train_loss: 0.01734553419890395
FOLD: 0, EPOCH: 10, valid_loss: 0.017063785743500506
FOLD: 0, EPOCH: 11, train_loss: 0.01738146605456005
FOLD: 0, EPOCH: 11, valid_loss: 0.016972185165754387
FOLD: 0, EPOCH: 12, train_loss: 0.01726998638469672
FOLD: 0, EPOCH: 12, valid_loss: 0.017069742163377147
FOLD: 0, EPOCH: 13, train_loss: 0.01716839997232824
FOLD: 0, EPOCH: 13, valid_loss: 0.017163482893790518
FOLD: 0, EPOCH: 14, train_loss: 0.017145884688943624
FOLD: 0, EPOCH: 14, valid_loss: 0.01688227863716228
FOLD: 0, EPOCH: 15, train_loss: 0.01695431390291323
FOLD: 0, EPOCH: 15, valid_loss: 0.01666050689028842
FOLD: 0, EPOCH: 16, train_loss: 0.016806445307219805
FOLD: 0,

FOLD: 3, EPOCH: 11, train_loss: 0.017286503032875666
FOLD: 3, EPOCH: 11, valid_loss: 0.017236891841249807
FOLD: 3, EPOCH: 12, train_loss: 0.017189651545461104
FOLD: 3, EPOCH: 12, valid_loss: 0.01716938064034496
FOLD: 3, EPOCH: 13, train_loss: 0.017089010722449293
FOLD: 3, EPOCH: 13, valid_loss: 0.01714643726923636
FOLD: 3, EPOCH: 14, train_loss: 0.017095137751944687
FOLD: 3, EPOCH: 14, valid_loss: 0.01695208695850202
FOLD: 3, EPOCH: 15, train_loss: 0.016944064056851726
FOLD: 3, EPOCH: 15, valid_loss: 0.0169090743043593
FOLD: 3, EPOCH: 16, train_loss: 0.016806757415010445
FOLD: 3, EPOCH: 16, valid_loss: 0.016844423008816584
FOLD: 3, EPOCH: 17, train_loss: 0.016637554621674877
FOLD: 3, EPOCH: 17, valid_loss: 0.016652442674551692
FOLD: 3, EPOCH: 18, train_loss: 0.01644166861323343
FOLD: 3, EPOCH: 18, valid_loss: 0.016567788299705302
FOLD: 3, EPOCH: 19, train_loss: 0.01619405388508154
FOLD: 3, EPOCH: 19, valid_loss: 0.01648238655179739
FOLD: 3, EPOCH: 20, train_loss: 0.015956269590643005
F

In [36]:
train_targets_scored

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
len(target_cols)

206

In [38]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)


CV log_loss:  0.014655359766323124


In [39]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [40]:
sub.shape

(3982, 207)